# Exploratory Data Analysis - Bank Churn Dataset

This notebook performs comprehensive exploratory data analysis on the bank churn dataset to understand:
- Dataset structure and data quality
- Distribution of features
- Relationships between features and churn
- Key insights for model development

## Table of Contents
1. [Data Loading and Initial Inspection](#1-data-loading-and-initial-inspection)
2. [Data Quality Assessment](#2-data-quality-assessment)
3. [Target Variable Analysis](#3-target-variable-analysis)
4. [Univariate Analysis](#4-univariate-analysis)
5. [Bivariate Analysis](#5-bivariate-analysis)
6. [Correlation Analysis](#6-correlation-analysis)
7. [Key Insights and Conclusions](#7-key-insights-and-conclusions)


In [ ]:
# Import necessary libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# Add src directory to path for imports
sys.path.append(str(Path('../').resolve()))
from src.utils import (
    load_data, get_data_overview, get_statistical_summary,
    check_class_imbalance, plot_target_distribution,
    plot_numeric_distributions, plot_categorical_analysis,
    plot_correlation_matrix, get_feature_importance_correlation,
    validate_data_quality
)

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"Working directory: {os.getcwd()}")


## 1. Data Loading and Initial Inspection


In [ ]:
# Load the dataset
data_path = '../data/Churn Modeling.csv'
df = load_data(data_path)

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()


In [ ]:
# Get comprehensive overview
overview = get_data_overview(df)

print("=" * 60)
print("DATASET OVERVIEW")
print("=" * 60)
print(f"Shape: {overview['shape']} (rows, columns)")
print(f"\nColumns ({len(overview['columns'])}):")
for col in overview['columns']:
    print(f"  - {col}")
print(f"\nMemory Usage: {overview['memory_usage_mb']:.2f} MB")
print(f"\nDuplicate Rows: {overview['duplicate_rows']}")
print(f"\nNumeric Columns: {overview['numeric_columns']}")
print(f"\nCategorical Columns: {overview['categorical_columns']}")


In [ ]:
# Display data types and basic info
print("=" * 60)
print("DATA TYPES")
print("=" * 60)
print(df.dtypes)
print("\n" + "=" * 60)
print("DATA INFO")
print("=" * 60)
df.info()


In [ ]:
# Display descriptive statistics
print("=" * 60)
print("DESCRIPTIVE STATISTICS")
print("=" * 60)
df.describe()


## 2. Data Quality Assessment


In [ ]:
# Validate data quality
quality_issues = validate_data_quality(df, target_col='Exited')

print("=" * 60)
print("DATA QUALITY ASSESSMENT")
print("=" * 60)

# Check for missing values
if quality_issues['missing_values']:
    print("\nMissing Values Found:")
    for col, count in quality_issues['missing_values'].items():
        percentage = (count / len(df)) * 100
        print(f"  - {col}: {count} ({percentage:.2f}%)")
else:
    print("\n✓ No missing values found!")

# Check for duplicates
if quality_issues['duplicates'] > 0:
    print(f"\n⚠ Warning: {quality_issues['duplicates']} duplicate rows found")
else:
    print("\n✓ No duplicate rows found!")

# Check target variable
if quality_issues['invalid_target_values']:
    print(f"\n⚠ {quality_issues['invalid_target_values']}")
else:
    print("\n✓ Target variable values are valid (0 or 1)")

# Check for outliers
if quality_issues['outliers']:
    print("\nOutliers detected (using IQR method):")
    for col, count in quality_issues['outliers'].items():
        percentage = (count / len(df)) * 100
        print(f"  - {col}: {count} outliers ({percentage:.2f}%)")
else:
    print("\n✓ No significant outliers detected")


## 3. Target Variable Analysis


In [ ]:
# Analyze target variable distribution
target_analysis = check_class_imbalance(df, target_col='Exited')

print("=" * 60)
print("TARGET VARIABLE ANALYSIS")
print("=" * 60)
print(f"\nClass Distribution:")
for class_val, count in target_analysis['counts'].items():
    prop = target_analysis['proportions'][class_val]
    label = "Churned" if class_val == 1 else "Retained"
    print(f"  {label} ({class_val}): {count} ({prop:.2%})")

print(f"\nImbalance Ratio: {target_analysis['imbalance_ratio']:.3f}")
if target_analysis['is_imbalanced']:
    print("⚠ Dataset is imbalanced - consider using techniques like SMOTE or class weights")
else:
    print("✓ Dataset is relatively balanced")


In [ ]:
# Visualize target distribution
fig = plot_target_distribution(df, target_col='Exited', figsize=(12, 5))
plt.show()


## 4. Univariate Analysis

### 4.1 Numerical Features Distribution


In [ ]:
# Get statistical summary by target variable
summary_stats = get_statistical_summary(df, target_col='Exited')
print("=" * 60)
print("STATISTICAL SUMMARY BY CHURN STATUS")
print("=" * 60)
summary_stats


In [ ]:
# Plot distributions of numeric features by churn status
fig = plot_numeric_distributions(df, target_col='Exited', figsize=(18, 12))
plt.show()


In [ ]:
# Detailed analysis of key numerical features
key_features = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary', 'Tenure', 'NumOfProducts']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, feature in enumerate(key_features):
    ax = axes[idx]
    
    # Create histogram with KDE
    for exited in [0, 1]:
        label = 'Retained' if exited == 0 else 'Churned'
        data = df[df['Exited'] == exited][feature]
        ax.hist(data, alpha=0.6, label=label, bins=30, density=True)
    
    ax.set_xlabel(feature)
    ax.set_ylabel('Density')
    ax.set_title(f'Distribution of {feature} by Churn Status')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### 4.2 Categorical Features Analysis


In [ ]:
# Analyze categorical features
categorical_cols = ['Geography', 'Gender']
fig = plot_categorical_analysis(df, categorical_cols, target_col='Exited', figsize=(14, 5))
plt.show()


In [ ]:
# Detailed categorical analysis with counts
print("=" * 60)
print("CATEGORICAL FEATURES ANALYSIS")
print("=" * 60)

for col in categorical_cols:
    print(f"\n{col}:")
    print("-" * 40)
    crosstab = pd.crosstab(df[col], df['Exited'], margins=True)
    crosstab_pct = pd.crosstab(df[col], df['Exited'], normalize='index') * 100
    print(crosstab)
    print(f"\nChurn Rates by {col}:")
    for val in df[col].unique():
        churn_rate = df[df[col] == val]['Exited'].mean()
        count = len(df[df[col] == val])
        print(f"  {val}: {churn_rate:.2%} (n={count})")


In [ ]:
# Analyze binary features
binary_features = ['HasCrCard', 'IsActiveMember']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, feature in enumerate(binary_features):
    ax = axes[idx]
    
    # Calculate churn rates
    churn_rates = df.groupby(feature)['Exited'].agg(['mean', 'count'])
    
    # Create bar plot
    bars = ax.bar(churn_rates.index.astype(str), churn_rates['mean'], 
                  color=sns.color_palette('viridis', len(churn_rates)))
    ax.set_xlabel(feature)
    ax.set_ylabel('Churn Rate')
    ax.set_title(f'Churn Rate by {feature}')
    ax.set_ylim([0, 1])
    
    # Add labels
    for i, (bar, count) in enumerate(zip(bars, churn_rates['Count'])):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{height:.2%}\n(n={count})',
               ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()


## 5. Bivariate Analysis

### 5.1 Age Groups Analysis


In [ ]:
# Create age groups
df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 30, 40, 50, 60, 100], 
                        labels=['<30', '30-40', '40-50', '50-60', '60+'])

# Analyze churn by age group
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Churn rate by age group
churn_by_age = df.groupby('AgeGroup')['Exited'].agg(['mean', 'count'])
bars = axes[0].bar(range(len(churn_by_age)), churn_by_age['mean'],
                   color=sns.color_palette('viridis', len(churn_by_age)))
axes[0].set_xticks(range(len(churn_by_age)))
axes[0].set_xticklabels(churn_by_age.index)
axes[0].set_ylabel('Churn Rate')
axes[0].set_title('Churn Rate by Age Group')
axes[0].set_ylim([0, 1])

for i, (bar, count) in enumerate(zip(bars, churn_by_age['Count'])):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2%}\n(n={count})',
                ha='center', va='bottom', fontsize=9)

# Distribution of age by churn
for exited in [0, 1]:
    label = 'Retained' if exited == 0 else 'Churned'
    axes[1].hist(df[df['Exited'] == exited]['Age'], alpha=0.6, 
                 label=label, bins=20)
axes[1].set_xlabel('Age')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Age Distribution by Churn Status')
axes[1].legend()

plt.tight_layout()
plt.show()

print("Churn Rates by Age Group:")
for age_group in df['AgeGroup'].cat.categories:
    churn_rate = df[df['AgeGroup'] == age_group]['Exited'].mean()
    count = len(df[df['AgeGroup'] == age_group])
    print(f"  {age_group}: {churn_rate:.2%} (n={count})")


### 5.2 Balance Analysis


In [ ]:
# Analyze balance and its relationship with churn
print("=" * 60)
print("BALANCE ANALYSIS")
print("=" * 60)

# Check zero balance customers
zero_balance = df[df['Balance'] == 0]
zero_balance_churn = zero_balance['Exited'].mean()
print(f"\nCustomers with Zero Balance: {len(zero_balance)} ({len(zero_balance)/len(df):.2%})")
print(f"Churn Rate (Zero Balance): {zero_balance_churn:.2%}")

non_zero_balance = df[df['Balance'] > 0]
non_zero_balance_churn = non_zero_balance['Exited'].mean()
print(f"\nCustomers with Non-Zero Balance: {len(non_zero_balance)} ({len(non_zero_balance)/len(df):.2%})")
print(f"Churn Rate (Non-Zero Balance): {non_zero_balance_churn:.2%}")

# Create balance groups
df['BalanceGroup'] = pd.cut(df['Balance'], 
                           bins=[-1, 0, 50000, 100000, 150000, float('inf')],
                           labels=['0', '0-50K', '50K-100K', '100K-150K', '150K+'])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Churn rate by balance group
churn_by_balance = df.groupby('BalanceGroup')['Exited'].agg(['mean', 'count'])
bars = axes[0].bar(range(len(churn_by_balance)), churn_by_balance['mean'],
                   color=sns.color_palette('viridis', len(churn_by_balance)))
axes[0].set_xticks(range(len(churn_by_balance)))
axes[0].set_xticklabels(churn_by_balance.index, rotation=45, ha='right')
axes[0].set_ylabel('Churn Rate')
axes[0].set_title('Churn Rate by Balance Group')
axes[0].set_ylim([0, 1])

for i, (bar, count) in enumerate(zip(bars, churn_by_balance['Count'])):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2%}\n(n={count})',
                ha='center', va='bottom', fontsize=8)

# Balance distribution
for exited in [0, 1]:
    label = 'Retained' if exited == 0 else 'Churned'
    data = df[df['Exited'] == exited]['Balance']
    axes[1].hist(data, alpha=0.6, label=label, bins=50)
axes[1].set_xlabel('Balance')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Balance Distribution by Churn Status')
axes[1].legend()

plt.tight_layout()
plt.show()


### 5.3 Product and Tenure Analysis


In [ ]:
# Analyze number of products and tenure
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Churn rate by number of products
churn_by_products = df.groupby('NumOfProducts')['Exited'].agg(['mean', 'count'])
bars1 = axes[0, 0].bar(churn_by_products.index, churn_by_products['mean'],
                       color=sns.color_palette('viridis', len(churn_by_products)))
axes[0, 0].set_xlabel('Number of Products')
axes[0, 0].set_ylabel('Churn Rate')
axes[0, 0].set_title('Churn Rate by Number of Products')
axes[0, 0].set_ylim([0, 1])
for bar, count in zip(bars1, churn_by_products['Count']):
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2%}\n(n={count})',
                   ha='center', va='bottom', fontsize=9)

# Churn rate by tenure
churn_by_tenure = df.groupby('Tenure')['Exited'].agg(['mean', 'count'])
bars2 = axes[0, 1].bar(churn_by_tenure.index, churn_by_tenure['mean'],
                       color=sns.color_palette('viridis', len(churn_by_tenure)))
axes[0, 1].set_xlabel('Tenure (Years)')
axes[0, 1].set_ylabel('Churn Rate')
axes[0, 1].set_title('Churn Rate by Tenure')
axes[0, 1].set_ylim([0, 1])
for bar, count in zip(bars2, churn_by_tenure['Count']):
    height = bar.get_height()
    axes[0, 1].text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2%}',
                   ha='center', va='bottom', fontsize=7, rotation=90)

# Distribution of products
for exited in [0, 1]:
    label = 'Retained' if exited == 0 else 'Churned'
    data = df[df['Exited'] == exited]['NumOfProducts']
    axes[1, 0].hist(data, alpha=0.6, label=label, bins=range(1, 6), align='left')
axes[1, 0].set_xlabel('Number of Products')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Number of Products Distribution')
axes[1, 0].legend()

# Distribution of tenure
for exited in [0, 1]:
    label = 'Retained' if exited == 0 else 'Churned'
    data = df[df['Exited'] == exited]['Tenure']
    axes[1, 1].hist(data, alpha=0.6, label=label, bins=range(0, 12), align='left')
axes[1, 1].set_xlabel('Tenure (Years)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Tenure Distribution')
axes[1, 1].legend()

plt.tight_layout()
plt.show()


## 6. Correlation Analysis


In [ ]:
# Plot correlation matrix
fig = plot_correlation_matrix(df, target_col='Exited', figsize=(12, 10))
plt.show()


In [ ]:
# Get feature importance based on correlation with target
feature_corr = get_feature_importance_correlation(df, target_col='Exited')

print("=" * 60)
print("FEATURE CORRELATION WITH TARGET (Exited)")
print("=" * 60)
print(feature_corr.to_string(index=False))

# Visualize feature correlations
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(feature_corr['Feature'], feature_corr['Correlation'],
               color=sns.color_palette('viridis', len(feature_corr)))
ax.set_xlabel('Absolute Correlation with Churn')
ax.set_title('Feature Importance (Correlation with Target)')
ax.invert_yaxis()

for i, (bar, corr) in enumerate(zip(bars, feature_corr['Correlation'])):
    ax.text(corr, bar.get_y() + bar.get_height()/2,
           f'{corr:.3f}',
           ha='left', va='center', fontsize=9)

plt.tight_layout()
plt.show()


## 7. Key Insights and Conclusions

### Summary of Findings


In [ ]:
# Generate summary insights
print("=" * 60)
print("KEY INSIGHTS FROM EDA")
print("=" * 60)

# 1. Dataset overview
print("\n1. DATASET CHARACTERISTICS:")
print(f"   - Total records: {len(df):,}")
print(f"   - Features: {len(df.columns) - 1} (excluding target)")
print(f"   - No missing values detected")
print(f"   - No duplicate records")

# 2. Target variable
print("\n2. TARGET VARIABLE (Churn):")
churn_rate = df['Exited'].mean()
print(f"   - Overall churn rate: {churn_rate:.2%}")
print(f"   - Churned customers: {df['Exited'].sum():,}")
print(f"   - Retained customers: {(df['Exited'] == 0).sum():,}")
if target_analysis['is_imbalanced']:
    print(f"   - ⚠ Dataset is imbalanced (ratio: {target_analysis['imbalance_ratio']:.3f})")

# 3. Geography
print("\n3. GEOGRAPHY:")
geo_churn = df.groupby('Geography')['Exited'].agg(['mean', 'count'])
for geo, row in geo_churn.iterrows():
    print(f"   - {geo}: {row['mean']:.2%} churn rate (n={row['count']:,})")

# 4. Gender
print("\n4. GENDER:")
gender_churn = df.groupby('Gender')['Exited'].agg(['mean', 'count'])
for gender, row in gender_churn.iterrows():
    print(f"   - {gender}: {row['mean']:.2%} churn rate (n={row['count']:,})")

# 5. Age
print("\n5. AGE:")
age_churn = df.groupby('AgeGroup')['Exited'].agg(['mean', 'count'])
for age_group, row in age_churn.iterrows():
    print(f"   - {age_group}: {row['mean']:.2%} churn rate (n={row['count']:,})")

# 6. Active membership
print("\n6. ACTIVE MEMBERSHIP:")
active_churn = df.groupby('IsActiveMember')['Exited'].agg(['mean', 'count'])
for active, row in active_churn.iterrows():
    status = "Active" if active == 1 else "Inactive"
    print(f"   - {status}: {row['mean']:.2%} churn rate (n={row['count']:,})")

# 7. Balance
print("\n7. BALANCE:")
zero_bal_churn = df[df['Balance'] == 0]['Exited'].mean()
non_zero_bal_churn = df[df['Balance'] > 0]['Exited'].mean()
print(f"   - Zero balance: {zero_bal_churn:.2%} churn rate")
print(f"   - Non-zero balance: {non_zero_bal_churn:.2%} churn rate")

# 8. Top correlated features
print("\n8. TOP FEATURES CORRELATED WITH CHURN:")
top_features = feature_corr.head(5)
for _, row in top_features.iterrows():
    print(f"   - {row['Feature']}: {row['Correlation']:.3f}")

print("\n" + "=" * 60)
print("RECOMMENDATIONS FOR MODELING")
print("=" * 60)
print("1. Handle class imbalance using SMOTE or class weights")
print("2. Consider feature engineering for Age (age groups) and Balance (balance groups)")
print("3. Encode categorical variables (Geography, Gender)")
print("4. Scale numerical features for distance-based algorithms")
print("5. Focus on top correlated features: Age, Balance, IsActiveMember, Geography")
print("6. Consider interaction features (e.g., Age × Balance, Geography × IsActiveMember)")


In [ ]:
# Save cleaned dataset with engineered features for next phase
df_cleaned = df.copy()

# Save to CSV (optional - for use in next phase)
# df_cleaned.to_csv('../data/churn_data_cleaned.csv', index=False)
# print("Cleaned dataset saved successfully!")

print("EDA Complete! Ready for Phase 3: Data Preprocessing & Feature Engineering")
